<a href="https://colab.research.google.com/github/imrealhelper/Linear-Algebra/blob/main/Computational_2024_mid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6DOF Ballistic simulation

__<div style="text-align: right"> ASE3001: Computational Experiments for Aerospace Engineering, Inha University. </div>__
_<div style="text-align: right"> Jong-Han Kim (jonghank@inha.ac.kr) </div>_



**서약:** 나는 정직하게 시험에 응할 것을 서약합니다.

---

비행체의 운동을 기술하기 위해 다음과 같은 14개의 상태변수를 사용한다.

*   **Body-fixed 좌표계 속도:** $u, v, w$ (m/s) - 각각 전진, 우측, 하방 속도
*   **Body-fixed 좌표계 각속도:** $p, q, r$ (rad/s) - 각각 롤, 피치, 요 각속도
*   **관성 좌표계 위치:** $x, y, h$ (m) - 각각 동쪽(East), 북쪽(North), 고도(Altitude, Up)
*   **공력 각:** $\alpha, \beta$ (rad) - 각각 받음각(Angle of Attack), 옆미끄럼각(Sideslip Angle)
*   **오일러 각 (자세):** $\psi, \theta, \phi$ (rad) - 각각 요(Yaw), 피치(Pitch), 롤(Roll) 각 (Z-Y-X 순서)

비행체의 6자유도 강체 운동방정식은 다음과 같이 주어진다.

**가. 병진 운동 방정식 (Body-fixed 좌표계):**
\begin{align*} m(\dot{u} - rv + qw) &= X - mg \sin\theta \\ m(\dot{v} - pw + ru) &= Y + mg \cos\theta \sin\phi \\ m(\dot{w} - qu + pv) &= Z + mg \cos\theta \cos\phi \end{align*}

**나. 회전 운동 방정식 (Body-fixed 좌표계):**
\begin{align*} I_{xx}\dot{p} - (I_{yy} - I_{zz})qr &= L \\ I_{yy}\dot{q} - (I_{zz} - I_{xx})rp &= M \\ I_{zz}\dot{r} - (I_{xx} - I_{yy})pq &= N \end{align*}

**다. 운동학 방정식:**
\begin{align*} \dot{x} &= u\cos\theta\cos\psi + v(\sin\phi\sin\theta\cos\psi - \cos\phi\sin\psi) + w(\cos\phi\sin\theta\cos\psi + \sin\phi\sin\psi) \\ \dot{y} &= u\cos\theta\sin\psi + v(\sin\phi\sin\theta\sin\psi + \cos\phi\cos\psi) + w(\cos\phi\sin\theta\sin\psi - \sin\phi\cos\psi) \\ \dot{h} &= u\sin\theta - v\sin\phi\cos\theta - w\cos\phi\cos\theta \\ \dot{\phi} &= p + (q\sin\phi + r\cos\phi)\tan\theta \\ \dot{\theta} &= q\cos\phi - r\sin\phi \\ \dot{\psi} &= (q\sin\phi + r\cos\phi)\sec\theta \end{align*}

여기서 $X, Y, Z$는 body-fixed 좌표계 기준 공력 성분이며, $L, M, N$은 body-fixed 좌표계 기준 공력 모멘트 성분이다. $m$은 질량, $I_{xx}, I_{yy}, I_{zz}$는 관성 모멘트, $g$는 중력 가속도이다.

**라. 공력 각 및 총 속도:**
\begin{align*} V &= \sqrt{u^2 + v^2 + w^2} \\ \alpha &= \arctan(w/u) \\ \beta &= \arcsin(v/V) \end{align*}

**마. 공력 각 변화율:**
\begin{align*} \dot{V} &= (u\dot{u} + v\dot{v} + w\dot{w}) / V \\ \dot{\alpha} &= (u\dot{w} - w\dot{u}) / (u^2 + w^2) \\ \dot{\beta} &= (V\dot{v} - v\dot{V}) / (V\sqrt{u^2 + w^2}) \end{align*}


**2. 환경 모델**

*   **대기 밀도 ($\rho$):** 고도 $h$(m)의 함수로 다음과 같이 모델링한다 ($\text{kg/m}^3$).

    $$\rho(h) = 1.225 \exp(-h/8500)$$
*   **중력 가속도 ($g$):** 고도 $h$(m)의 함수로 다음과 같이 모델링한다 ($\text{m/s}^2$). 지구 반경 $R_e = 6378.137 \times 1000$ m.

    $$ g(h) = 9.80665 (R_e / (R_e + h))^2 $$

**3. 공력/모멘트 모델**

동압 $q_\infty = \frac{1}{2}\rho V^2$ 이다. 공력 $F = (X, Y, Z)$ 와 공력 모멘트 $\mathcal{M} = (L, M, N)$ 는 다음과 같이 계산된다. $S$는 기준 면적, $d$는 기준 길이이다.

*   **Body-fixed 축력 계수 ($C_X$):** $C_X = C_{X,0} + C_{X,\alpha^2} \alpha^2 + C_{X,\beta^2} \beta^2$
*   **Body-fixed 측력 계수 ($C_Y$):** $C_Y = C_{Y,0} + C_{Y,\beta} \beta$
*   **Body-fixed 수직력 계수 ($C_Z$):** $C_Z = C_{Z,0} + C_{Z,\alpha} \alpha$
*   **롤링 모멘트 계수 ($C_l$):** $C_l = C_{l,p} \frac{pd}{2V}$
*   **피칭 모멘트 계수 ($C_m$):** $C_m = C_{m,0} + C_{m,\alpha} \alpha + C_{m,q} \frac{qd}{2V}$
*   **요잉 모멘트 계수 ($C_n$):** $C_n = C_{n,0} + C_{n,\beta} \beta + C_{n,r} \frac{rd}{2V}$

공력 및 모멘트는 다음과 같이 계산한다.
\begin{align*} X &= q_\infty S C_X \\ Y &= q_\infty S C_Y \\ Z &= q_\infty S C_Z \\ L &= q_\infty S d C_l \\ M &= q_\infty S d C_m \\ N &= q_\infty S d C_n \end{align*}

**4. 파라미터**

*   **질량 및 형상:**
    *   $m = 1000$ kg
    *   $l = 8$ m (길이 - 모델 계산에는 직접 사용되지 않음)
    *   $d = 0.8$ m (직경/기준길이)
    *   $S = 0.5$ m$^2$ (기준면적)
*   **관성 모멘트:**
    *   $I_{xx} = 300$ kg$\cdot$m$^2$
    *   $I_{yy} = 19000$ kg$\cdot$m$^2$
    *   $I_{zz} = 21000$ kg$\cdot$m$^2$
*   **공력 계수:**
    *   $C_{X,0} = -0.2$
    *   $C_{X,\alpha^2} = -2.1$
    *   $C_{X,\beta^2} = -1.7$
    *   $C_{Y,0} = 0$
    *   $C_{Y,\beta} = -2.3$
    *   $C_{Z,0} = -0.02$
    *   $C_{Z,\alpha} = -3.8$
*   **공력 모멘트 계수:**
    *   $C_{l,p} = -0.01$
    *   $C_{m,0} = 0.02$
    *   $C_{m,\alpha} = -5$
    *   $C_{m,q} = -60$
    *   $C_{n,0} = 0$
    *   $C_{n,\beta} = 7$
    *   $C_{n,r} = -45$

**5. 초기 조건**

시간 $t=0$ 에서의 상태는 다음과 같다.
*   $h = 2000$ m
*   $V = 300$ m/s
*   $\alpha = 2^\circ$ (단위 변환 필요)
*   $\beta = 0^\circ$ (단위 변환 필요)
*   $\psi = 80^\circ$ (Yaw, 단위 변환 필요)
*   $\theta = 20^\circ$ (Pitch, 단위 변환 필요)
*   $\phi = 0^\circ$ (Roll, 단위 변환 필요)
*   $p = 2^\circ$/s (단위 변환 필요)
*   $q = 10^\circ$/s (단위 변환 필요)
*   $r = -5^\circ$/s (단위 변환 필요)
*   $x = 0$ m
*   $y = 0$ m

*(초기 $u, v, w$는 $V, \alpha, \beta$로부터 계산해야 함)*

**6. 시뮬레이션 요구사항**

**가. 상태 변수 미분 함수 작성:**
시간 `t`와 상태 벡터 `z = [u,v,w, p,q,r, x,y,h, alpha, beta, yaw, pitch, roll]` 를 입력받아 상태 변수의 시간 미분 벡터 $\dot{z}$를 계산하는 Python 함수 `model(z, t)`를 작성하시오. 함수 내에서 필요한 물리량(밀도, 중력, 동압, 공력/모멘트 등)을 계산해야 한다.

**나. 수치 적분 수행:**
작성된 `model` 함수와 주어진 초기 조건을 이용하여 $t=0$부터 $t=2000$초까지 비행체의 상태 변화를 수치적으로 적분하시오. 시간 간격 $\Delta t$는 100,000개의 시간 스텝을 갖도록 설정하시오. 비행체 고도 $h$가 0 이하가 되면 시뮬레이션을 종료하시오.

**다. 결과 분석 및 제출 :**

*   **(Problem a):** 시간에 따른 주요 상태 변수들의 변화를 나타내는 그래프들을 제출하시오.
    *   시간에 따른 속도(V) 변화
    *   시간에 따른 오일러 각($\psi, \theta, \phi$) 변화 (단위: rad)
    *   시간에 따른 body-axis 각속도($p, q, r$) 변화 (단위: rad/s)
    *   시간에 따른 받음각($\alpha$) 변화 (단위: rad)
    *   비행 궤적을 나타내는 그래프들을 제출하시오.
        *   지상 궤적 (East-North 평면, x vs y)
        *   고도 프로파일 (East-Altitude 평면, x vs h 또는 y vs h)
    *   각 그래프에는 축 레이블과 단위를 명확히 표시하시오. 모든 그래프는 마지막 시뮬레이션 종료 시점까지만 표시하시오.

*   **(Problem b):** 시뮬레이션 종료 시점(지면 충돌 시점)에서의 동쪽(East) 위치 $x$와 북쪽(North) 위치 $y$를 보고하시오 (단위: m).

*   **(Problem c):** 전체 비행 시간 동안의 최저 속도(V)를 보고하시오 (단위: m/s).

*   **(Problem d):** 피치 각($\theta$)과 요 각($\psi$)의 시간에 따른 변화에 대해 FFT(고속 푸리에 변환) 분석을 수행하여, 가장 지배적인(dominant) 5개의 주파수 성분을 각각 보고하시오 (단위: Hz). 각 주파수 성분의 크기(magnitude)도 함께 제시하시오. 주파수 스펙트럼 그래프를 제출하시오 (x축: 주파수(Hz), y축: 크기(Magnitude), x축 범위는 주요 성분이 잘 보이도록 적절히 조절).

---

In [5]:
##Your answer here.

